# LLM

# TaxRec

In [ ]:

!pip install -qU \
  torch\
  torchvision\

!pip install -qU \
  transformers\
  sentence-transformers\
  huggingface-hub \
  recbole \
  lightfm \
  tqdm \
  langchain-community \
  gensim \



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#──────────────────────────────────────────────────────────
# 3. item.csv  ➜  ml100k.item  (BERT-признаки)
#──────────────────────────────────────────────────────────
# Файл вида:
# item_id,title,Genre,Themes,Streaming Availability,IMDb,Rotten Tomatoes,Metacritic,MPAA Rating,Language,Release Year,Critical Reception
import os, json, re, ast, numpy as np, pandas as pd, torch
from tqdm import tqdm
from lightfm.datasets import fetch_movielens
from sentence_transformers import SentenceTransformer

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import SASRec, BERT4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, set_color

#──────────────────────────────────────────────────────────
# 2. Подготовка MovieLens-100K  ➜  ml100k.inter
#──────────────────────────────────────────────────────────
ml = fetch_movielens()                      # 943 users × 1682 movies
train_mat, test_mat = ml['train'], ml['test']
full = train_mat + test_mat                # объединяем, т.к. ML100k без timestamps

coo = full.tocoo()
coo = full.tocoo()
df_inter = pd.DataFrame({
    'user_id:token'  : coo.row.astype(int),
    'item_id:token'  : coo.col.astype(int),
    'timestamp:float': coo.col.astype(int),    # фиктивные ts = item_id
    'class:float'    : np.ones(coo.row.shape, dtype=int),
    'label:float'    : np.ones(coo.row.shape, dtype=float) # все взаимодействия – положительные
})

data_dir = './ml100k'
os.makedirs(data_dir, exist_ok=True)
inter_path = os.path.join(data_dir, 'ml100k.inter')
df_inter.to_csv(inter_path, index=False)
print('✓ .inter сохранён:', inter_path, df_inter.shape)

csv_path = '/content/drive/MyDrive/item.csv'  
df_meta  = pd.read_csv(csv_path)

df_meta = df_meta.sort_values('item_id').reset_index(drop=True)
titles = ml['item_labels']
assert len(df_meta) == full.shape[1], "item.csv и MovieLens имеют разную длину"

cols = ['Genre','Themes','Streaming Availability','IMDb',
        'Rotten Tomatoes','Metacritic','MPAA Rating',
        'Language','Release Year','Critical Reception']

sentences = []
for i, row in df_meta.iterrows():          # индекс == item_id - 1
    meta_str = ' | '.join(
        f'{c}: {row[c]}' for c in cols if pd.notna(row[c]) and row[c] != 'Unknown'
    )
    title = row['title']
    sentences.append(f"{title}. {meta_str}")

# ---------- эмбеддинги ----------
print('Sentence-BERT encoding…')
sbert = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
emb   = sbert.encode(sentences, batch_size=256, show_progress_bar=True,
                     normalize_embeddings=True)        # shape (1682, 384)


df_item = pd.DataFrame({
    'item_id:token'      : np.arange(len(titles), dtype=int),   
    'bert_emb:float_seq' : [' '.join(f'{x:.6f}' for x in v) for v in emb]
})
item_path = os.path.join(data_dir, 'ml100k.item')
df_item.to_csv(item_path, index=False)


In [ ]:
import numpy as np
import torch, inspect
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
np.float_ = np.float64
np.complex_ = np.complex128
np.unicode_ = np.str_
np.bool8 = np.bool_
sig = inspect.signature(torch.load)
if 'weights_only' in sig.parameters:
    _orig_load = torch.load
    def _torch_load_with_weights(f, *args, **kwargs):
        # если пользователь не указал weights_only явно — ставим False
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return _orig_load(f, *args, **kwargs)
    torch.load = _torch_load_with_weights


In [ ]:
import torch, os, warnings, logging

# 1) TF32 для GEMM
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32       = True
torch.set_float32_matmul_precision('high')

# 2) Тихий вывод
warnings.filterwarnings('ignore', category=FutureWarning)
logging.getLogger('recbole').setLevel(logging.ERROR)
os.environ['TORCH_CUDA_SHOW_DEPRECATION_WARNINGS'] = '0'


In [ ]:
import os, json, re, ast, numpy as np, pandas as pd, torch
from tqdm import tqdm
from lightfm.datasets import fetch_movielens
from sentence_transformers import SentenceTransformer

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import SASRec, BERT4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, set_color
import torch
torch.backends.cuda.matmul.allow_tf32 = True

In [ ]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
from recbole.model.context_aware_recommender import NFM, DeepFM
from recbole.model.sequential_recommender import GRU4RecF
def run_model(model_cls, use_feat=False, epochs=20, seed=42):
    from recbole.utils import init_seed
    init_seed(seed, True)

    cfg = {
        'USER_ID_FIELD' : 'user_id',
        'ITEM_ID_FIELD' : 'item_id',
        'TIME_FIELD'    : 'timestamp',
        'loss': 'bpr',
        'dataset'       : 'ml100k',
        'data_path'     : './',
        'field_separator': ',',                # ← CSV через “,”

        'load_col': {
            'inter': ['user_id', 'item_id', 'timestamp', 'class', 'label']
        },
        'device'      : 'cuda',
        'amp': True,
        'MAX_ITEM_LIST_LENGTH': 50,
        'seq_len': {},
        'n_epochs'      : epochs,
        'train_batch_size': 8192,
        'eval_batch_size' : 16384,
        'valid_metric'  : 'Recall@10',
        'metrics'       : ['Recall', 'NDCG'],
        'topk'          : 10,
        'seed'          : seed,
        'num_workers': 8,
        'eval_step'          : 0,
        'save_dataset'       : True,
        'CLASS_FIELD'    : 'class',
        'LABEL_FIELD'    : 'label',

        'neg_sampling' : None,
        'eval_type': 'ranking',
        'train_neg_sample_args': {
        'strategy': 'random',
        'sample_num': 1
    }    ,
    'eval_args': {
    'split':    {'RS': [0.8, 0.1, 0.1]},  # train=80%, valid=10%, test=10%
    'order':    'TO',
    'group_by': 'user'
}        # CE-loss → без нег.-семплинга
    }

    if use_feat:                               # подключаем BERT-вектора
        cfg['ITEM_FEATURE_FIELD'] = 'bert_emb'
        cfg['load_col']['item']   = ['item_id', 'bert_emb']

    config   = Config(model=model_cls, config_dict=cfg)
    dataset  = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    model   = model_cls(config, train_data.dataset).to(config['device'])
    trainer = Trainer(config, model)
    trainer.fit(train_data, valid_data, verbose=False)
    return trainer.evaluate(test_data, load_best_model=True)


#──────────────────────────────────────────────────────────
# 5. Запуск экспериментов
#──────────────────────────────────────────────────────────


experiments = {
    # 'GRU4RecF'     : (GRU4RecF, False),
    # 'GRU4RecF+Tax' : (GRU4RecF, True),
    'NFM'          : (NFM, False),
    'NFM+Tax'      : (NFM, True),
    'DeepFM'       : (DeepFM, False),
    'DeepFM+Tax'   : (DeepFM, True),
    'SasRec'   : (SASRec, False),
    'SasRec+Tax'   : (SASRec, True),
    'BERT4Rec'   : (BERT4Rec, False),
    'BERT4Rec+Tax'   : (BERT4Rec, True),

    # 'DIN'          : (DIN, False),
    # 'DIN+Tax'      : (DIN, True),
    # 'AutoInt'      : (AutoInt, False),
    # 'AutoInt+Tax'  : (AutoInt, True),
}

results = {}
for name, (cls, feat) in tqdm(experiments.items()):
    print(set_color(f'\n▶ {name}', 'green'))
    results[name] = run_model(cls, use_feat=feat, epochs=20)
    print(results[name])

#──────────────────────────────────────────────────────────
# 6. Таблица Recall@10 / NDCG@10
#──────────────────────────────────────────────────────────
tbl = pd.DataFrame(results).T[['Recall@10', 'NDCG@10']]


In [ ]:
print('\n===== ML100k: итог =====')
print(tbl.to_string(float_format='%.4f'))


In [ ]:
tbl.to_csv('res.csv')